# Age of Empires II Voobly Matches Data Exploration

## Load libraries and set options/environment

In [ ]:
# System
import sys, os, datetime
from datetime import date, timedelta

# DB connector
import pyodbc, mysql.connector
from sqlalchemy import create_engine

# Data Wrangling
import pandas as pd, numpy as np, re
from functools import reduce

# Visualization
import seaborn as sb, plotly, matplotlib.pyplot as plt

# Set options
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Read Data

In [ ]:
server = 'localhost'
database = 'AOE2_VOOBLY'
user = 'admin'
password = 'admin'

db_conn = mysql.connector.connect(host = server,
                                  port = 3306,
                                  user = user,
                                  password = password)

engine = create_engine("mysql+mysqlconnector://" + user + ":" + password + "@" + server + "/" + database)

In [ ]:
voobly_match_data = pd.read_sql('select * from aoe2_voobly.match_data', db_conn)